In [5]:
%load_ext autoreload
%autoreload 2
    
import os, sys
#PWD = os.getenv('PWD')
PWD = '/Users/chris/Documents/lab/emAnalysis/backend/server'

PROJ_MISSING_MSG = """Set an enviroment variable:\n
`DJANGO_PROJECT=your_project_name`\n
or call:\n
`init_django(your_project_name)`
"""

def init_django(project_name=None):
    os.chdir(PWD)
    project_name = project_name or os.environ.get('DJANGO_PROJECT') or None
    if project_name == None:
        raise Exception(PROJ_MISSING_MSG)
    sys.path.insert(0, os.getenv('PWD'))
    os.environ.setdefault('DJANGO_SETTINGS_MODULE', f'{project_name}.settings')
    os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
    import django
    django.setup()

init_django("server")
from django.conf import settings
media_root = settings.MEDIA_ROOT
import numpy as np
import os
from matplotlib import pyplot as plt
import functools
import math
import sims
import copy
import os
from PIL import Image
from pathlib import Path
import numpy as np
from skimage.measure import regionprops
from scipy import spatial
import pickle
from matplotlib.figure import Figure
import pyvips
from image.models import Image
from core.models import Canvas
from mims.models import MIMSImage, MIMSImageSet, MIMSAlignment
from PIL import Image
from scipy.ndimage import rotate
import pprint
import sims
import cv2

In [6]:
"""Selected Mims Image Set: af036a54-30db-4bf1-8376-40c519b8b20a

EM 1: 16936.49, 4725.90
MIMS 1: 6228.02, 3372.54
EM 2: 22969.22, 19449.35
MIMS 2: 5051.59, 3590.54
EM 3: 15669.92, 36195.05
MIMS 3: 4271.25, 4702.36
"""

'Selected Mims Image Set: af036a54-30db-4bf1-8376-40c519b8b20a\n\nEM 1: 16936.49, 4725.90\nMIMS 1: 6228.02, 3372.54\nEM 2: 22969.22, 19449.35\nMIMS 2: 5051.59, 3590.54\nEM 3: 15669.92, 36195.05\nMIMS 3: 4271.25, 4702.36\n'

In [15]:
import numpy as np
from skimage.transform import estimate_transform
import sims
from mims.models import MIMSImage, MIMSImageSet, MIMSAlignment
from PIL import Image
import math
from mims.model_utils import (
    get_concatenated_image,
    get_autocontrast_image_path,
)
import cv2
from PIL import ImageOps
import time

def largest_inner_square(side_length, angle):
    """Calculate the largest possible inner square side length for a given rotation angle."""
    # Convert angle to radians
    return side_length / (
        abs(math.cos(math.radians(angle))) + abs(math.sin(math.radians(angle)))
    )
def calculate_individual_mims_translations(mims_imageviewset, isotope):
    start = time.time()
    # Print seconds elapsed
    mims_images = mims_imageviewset.mims_images.all()
    # Get transformation parameters
    flip = mims_imageviewset.flip
    rotation_degrees = mims_imageviewset.rotation_degrees
    composite_image = Image.fromarray(
        get_concatenated_image(mims_imageviewset, isotope)
    )
    if isotope == "32S":
        composite_image = ImageOps.invert(composite_image)
    composite_image = composite_image.rotate(-rotation_degrees, expand=True)
    if flip:
        composite_image = composite_image.transpose(Image.FLIP_LEFT_RIGHT)
    composite_image = np.array(composite_image)
    em_image_obj = mims_imageviewset.canvas.images.first()
    scale = mims_imageviewset.pixel_size_nm / (em_image_obj.pixel_size_nm or 5)
    # Load the EM image as a numpy array
    em_image = Image.open(em_image_obj.file.path)
    em_image_array = np.array(em_image)
    scaled_em = cv2.resize(
        em_image_array,
        (int(em_image_array.shape[1] / scale), int(em_image_array.shape[0] / scale)),
    )
    # Calculate and store translations for each MIMS image
    for mims_image in mims_images[0:1]:
        filename = mims_image.file.name.split("/")[-1].split(".")[0]
        roi = filename.split("_")[-1]
        print(roi, "time1", time.time() - start)
        # Load the MIMS image for the isotope and load as PIL image
        img_orig = get_autocontrast_image_path(mims_image, isotope)
        img_orig = Image.open(img_orig)
        if isotope == "32S":
            img_orig = ImageOps.invert(img_orig)
        img = img_orig.copy()
        # Apply the flip and rotation if necessary to it
        img = img.rotate(-rotation_degrees, expand=True)
        if flip:
            img = img.transpose(Image.FLIP_LEFT_RIGHT)
        img_array = np.array(img)

        # Calculate the center of the image
        center_x, center_y = img_array.shape[1] // 2, img_array.shape[0] // 2

        # Calculate the cropping box
        largest_inner_square_side = int(
            largest_inner_square(img_orig.width, rotation_degrees)
        )
        start_x = center_x - largest_inner_square_side // 2
        start_y = center_y - largest_inner_square_side // 2
        end_x = center_x + largest_inner_square_side // 2
        end_y = center_y + largest_inner_square_side // 2

        # Crop the image to the largest inner square
        cropped_img_array = img_array[start_y:end_y, start_x:end_x]
        orig_cropped_img_array_shape = cropped_img_array.shape

        # Then find the image in the composite image
        result = cv2.matchTemplate(composite_image, img_array, cv2.TM_CCOEFF_NORMED)
        _, max_val, _, max_loc = cv2.minMaxLoc(result)
        # Now from this approximation, find a closer match in the actual EM image
        # Find the original EM location of the MIMS image
        print("hmm", max_loc, mims_imageviewset.canvas_y, mims_imageviewset.canvas_x, scale)
        mims_y = mims_imageviewset.canvas_y + max_loc[1] * scale
        mims_x = mims_imageviewset.canvas_x + max_loc[0] * scale
        mims_scaled_shape = np.array(img_orig).shape
        em_orig_shape = em_image_array.shape
        # Skip if sufficiently far outside of the known EM bounds
        approx_em_loc = [mims_y, mims_x]
        print("????", approx_em_loc, em_orig_shape, mims_scaled_shape)
        if (
            approx_em_loc[0] < -mims_scaled_shape[0]
            or approx_em_loc[0] > em_orig_shape[0] + mims_scaled_shape[0]
            or approx_em_loc[1] < -mims_scaled_shape[1]
            or approx_em_loc[1] > em_orig_shape[1] + mims_scaled_shape[1]
        ):
            print("Out of bounds: ", approx_em_loc, em_orig_shape)
            mims_image.status = "OUTSIDE_CANVAS"
            mims_image.save()
            continue
        print("Seems in bounds:", approx_em_loc, em_orig_shape)
        # Then scale it down
        mims_y_scaled = mims_y / scale
        mims_x_scaled = mims_x / scale
        crop_size = int(img_array.shape[0] * 1.3)
        amt_to_add_back = crop_size * 0.5
        crop_y_start = max(mims_y_scaled - amt_to_add_back, 0)
        crop_y_end = min(mims_y_scaled + (crop_size * 1.5), scaled_em.shape[0])
        crop_x_start = max(mims_x_scaled - amt_to_add_back, 0)
        crop_x_end = min(mims_x_scaled + (crop_size * 1.5), scaled_em.shape[1])
        scaled_em_crop = scaled_em[
            int(crop_y_start) : int(crop_y_end), int(crop_x_start) : int(crop_x_end)
        ]
        y_size_limit = int(scaled_em.shape[0] - mims_y_scaled)
        if (y_size_limit) < cropped_img_array.shape[0]:
            if crop_y_start == 0:
                cropped_img_array = cropped_img_array[-y_size_limit:, :]
            else:
                cropped_img_array = cropped_img_array[:y_size_limit, :]
        x_size_limit = int(scaled_em.shape[1] - mims_x_scaled)
        if (x_size_limit) < cropped_img_array.shape[1]:
            if crop_x_start == 0:
                cropped_img_array = cropped_img_array[:, -x_size_limit:]
            else:
                cropped_img_array = cropped_img_array[:, :x_size_limit]
        # Now find the MIMS image in the crop
        # These 2 calls take ~0.3 seconds
        if cropped_img_array.shape[0] == 0 or cropped_img_array.shape[1] == 0:
            mims_image.status = "NO_CELLS"
            mims_image.save()
            continue
        result = cv2.matchTemplate(
            scaled_em_crop, cropped_img_array, cv2.TM_CCOEFF_NORMED
        )
        _, _, _, max_loc = cv2.minMaxLoc(result)
        scaled_cropped_em_y_start = max(max_loc[1], 0)
        scaled_em_y_start = int(crop_y_start + scaled_cropped_em_y_start)
        em_y_start = int(scaled_em_y_start * scale)
        scaled_cropped_em_x_start = max(max_loc[0], 0)
        scaled_em_x_start = int(crop_x_start + scaled_cropped_em_x_start)
        em_x_start = int(scaled_em_x_start * scale)

        mims_image.alignments.all().delete()
        mims_image.alignments.create(
            x_offset=em_x_start,
            y_offset=em_y_start,
            flip_hor=flip,
            rotation_degrees=rotation_degrees,
            scale=scale,
            status="GUESS",
        )
        mims_image.status = "NEED_USER_ALIGNMENT"
        mims_image.save()
    return

viewset = MIMSImageSet.objects.get(id="af036a54-30db-4bf1-8376-40c519b8b20a")
calculate_individual_mims_translations(viewset, "SE")


0 time1 7.6144819259643555
hmm (3402, 2467) -22133 -7162 13.40419359524007
???? [10935.145599457253, 38439.06661100672] (47709, 34959) (512, 512)
Out of bounds:  [10935.145599457253, 38439.06661100672] (47709, 34959)


In [18]:
print(-22133+13.404*2467)
print(13.404*3402)

10934.667999999998
45600.408
